In [ ]:
# runs in jupyter container
import os
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset

DATA_DIR = os.getenv("MOVIELENS_DATA_DIR", "/mnt/Project-37")
print("Data dir:", DATA_DIR)

In [ ]:
# runs in jupyter container
ratings_path = os.path.join(DATA_DIR, "ml-192m", "ratings.csv")
df = pd.read_csv(ratings_path, nrows=100_00)
print(df.head())

In [ ]:
# runs in jupyter container
class MovieLensDataset(Dataset):
    def __init__(self, dataframe):
        self.users = torch.tensor(dataframe['userId'].values, dtype=torch.long)
        self.items = torch.tensor(dataframe['movieId'].values, dtype=torch.long)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx]

dataset = MovieLensDataset(df)
dataloader = DataLoader(dataset, batch_size=128, shuffle=False)


In [ ]:
# runs in jupyter container
model_path = "models/SSE_PT.pth"
model = torch.load(model_path, map_location="cpu")
model.eval()
print("Model loaded.")

In [ ]:
# Run a simple forward pass
with torch.no_grad():
    for i, (users, items) in enumerate(dataloader):
        outputs = model(users, items)
        print(f"Batch {i}, output shape: {outputs.shape}")
        if i >= 2:
            break